In [1]:
import requests
import re
import time
from tqdm import tqdm
from bs4 import BeautifulSoup

def read_chr_link(mw_category_group,href,title):
    chr_links=mw_category_group.findChildren(name='a')
    for i in range(len(chr_links)): 
        href.append('https://en.wikinews.org'+chr_links[i]['href'])
        title.append(chr_links[i].text)

def pagesearch(page_url,href,title):
    chr_contents=requests.get(page_url).text
    chr_soup= BeautifulSoup(chr_contents, 'html.parser')
    mw_pages=chr_soup.find(id="mw-pages")
    mw_category=mw_pages.findAll(name='div',attrs={"class":re.compile(r'mw-category')})[0]
    category_num=len(mw_category.findChildren(name='div',attrs={"class":re.compile(r'mw-category-group')}))
    mw_category_group=mw_pages.findAll(name='div',attrs={"class":re.compile(r'mw-category-group')})[0]
    if(category_num<=1):
        read_chr_link(mw_category_group,href,title)
        hreflinks=mw_pages.findAll(name='a',attrs={"title":re.compile(r'Category:Politics and conflicts')})
        if hreflinks[1]: next_page_url=re.sub(r'amp;','','https://en.wikinews.org'+hreflinks[1]['href'])
        elif hreflinks[3]: next_page_url=re.sub(r'amp;','','https://en.wikinews.org'+hreflinks[3]['href'])
        #print("category num:",category_num,",move to next page")
        pagesearch(next_page_url,href,title)
    else:
        read_chr_link(mw_category_group,href,title)
        #print("category num:",category_num,",not move to next page")
    return href

time_start=time.time()
r = requests.get('https://en.wikinews.org/wiki/Category:Politics_and_conflicts')
contents = r.text
news_row = []
for i in contents.split('<'):
    chrlink=re.match(r'a class=\"external text\" href=\"(.*)\">(.+)',i)
    if chrlink:
        if chrlink.group(2) in ['M','N','O','P','R','S','T','U','V','W']:
            href_list=[]; title_list=[]
            href_list=pagesearch(re.sub(r'amp;','',chrlink.group(1)),href_list,title_list)
            #print("char:",chrlink.group(2),"\tnum:",len(href_list))
            #print("first article:",href_list[0])
            #print("last article:",href_list[len(href_list)-1])
            for j in tqdm(range(len(href_list))):
                news_page = BeautifulSoup(requests.get(href_list[j]).text, 'html.parser').find(class_="mw-parser-output")
                date = [idate.get_text() for idate in news_page.findAll(name='strong',attrs={"class":re.compile(r'published')})]
                date = ''.join(date)
                #print(j,href_list[j])
                content = [icontent.get_text() for icontent in news_page.findAll(name='p')[1:]]
                content = ' '.join(content)
                #content = ''.join(content.split('\n'))
                content = content.replace('\n', ' ').replace('\r', ' ')
                news_row.append([chrlink.group(2),j+1,href_list[j],title_list[j],date,content])

time_end=time.time()
print('\ntime cost:',time_end-time_start,'s')
print("total article num:",len(news_row))

output = open('news1.xls','w',encoding='utf-8')
output.write('char\tnum\thref\ttitle\tdate\tcontent\n')
for i in range(len(news_row)):
	for j in range(len(news_row[i])):
		output.write(str(news_row[i][j]))
		output.write('\t')
	output.write('\n')
output.close()

#print(news_row)

100%|██████████| 326/326 [00:32<00:00,  9.97it/s]
time cost: 360.7176661491394 s
total article num: 3702

